In [145]:
import obspy
import sys
sys.path.append('/home/ak287/seisbench/seisbench/models')
import seisbench.models as sbm
from obspy.clients.fdsn import Client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from obspy.clients.fdsn.mass_downloader import Restrictions, MassDownloader, CircularDomain


client = Client('IRIS')

In [146]:
#model1 = sbm.QuakeXNetoneD.from_pretrained("base")
model2 = sbm.SeismicCNN.from_pretrained("base")

## We have a catalog of surface events

In [147]:
su_cat = pd.read_csv('../../data/event_catalogs_for_testing/surface_events_for_testing.csv')
su_cat.head(10)

,Date,Time,Lat,Lon,Depth,Mag,Magt,Nst,Gap,Clo,RMS,SRC,EventID,empty
0,2023/06/01,18:01:32.01,46.85080,-121.79300,-3.365,-5.0,Mh,0,0,0,0.00,UW,61929751,NaN
1,2023/06/01,18:39:58.07,46.87080,-121.73230,-2.877,-5.0,Mh,0,0,0,0.00,UW,61929771,NaN
2,2023/06/01,22:16:18.26,46.83560,-121.73300,-3.077,-5.0,Mh,0,0,0,0.00,UW,61929851,NaN
3,2023/06/03,05:11:55.95,46.83560,-121.73300,-3.077,-5.0,Mh,0,0,0,0.00,UW,61930241,NaN
4,2023/06/04,02:36:58.75,46.85080,-121.79300,-3.365,-5.0,Mh,0,0,0,0.00,UW,61930391,NaN
5,2023/06/05,06:37:01.29,45.35860,-121.70550,-2.590,-5.0,Mh,0,0,0,0.00,UW,61930566,NaN
6,2023/06/06,04:41:35.30,46.83560,-121.73300,-3.077,-5.0,Mh,0,0,0,0.00,UW,61930836,NaN
7,2023/06/06,09:12:18.56,48.75483,-121.83933,-0.500,1.1,ML,6,99,16,0.21,UW,61930856,NaN
8,2023/06/07,03:52:59.25,46.85080,-121.79300,-3.365,-5.0,Mh,0,0,0,0.00,UW,61931126,NaN
9,2023/06/07,05:24:16.48,46.83560,-121.73300,-3.077,-5.0,Mh,0,0,0,0.00,UW,61931136,NaN


In [148]:
# Combine Date and Time columns into a single datetime string
datetime_strs = su_cat['Date'] + ' ' + su_cat['Time']

# Convert each row to UTCDateTime
su_cat['UTCDateTime'] = datetime_strs.apply(obspy.UTCDateTime)


## Adding the station info to the catalog

In [74]:
# Add empty columns to store station info
su_cat['network'] = None
su_cat['station'] = None
su_cat['location'] = None
su_cat['channel_type'] = None

# Iterate through the DataFrame rows
for i, row in tqdm(su_cat.iterrows()):
    try:
        # Query station metadata near event location
        inv = client.get_stations(latitude=row['Lat'], 
                                  longitude=row['Lon'],
                                  starttime=row['UTCDateTime'] - 100,
                                  endtime=row['UTCDateTime'] + 100,  # add 1 second buffer
                                  maxradius=0.0001,
                                  level='response')
        
        # Assume first match is enough
        net = inv[0]
        sta = net[0]
        cha = sta[0]  # First channel (e.g., EHZ)

        su_cat.at[i, 'network'] = net.code
        su_cat.at[i, 'station'] = sta.code
        su_cat.at[i, 'location'] = cha.location_code
        su_cat.at[i, 'channel_type'] = cha.code[:2]  # 'EHZ' → 'EH'

    except Exception as e:
        print(f"Row {i} failed: {e}")
        continue


4it [00:00,  9.04it/s]

Row 2 failed: No data available for request.
HTTP Status code: 204
Detailed response of server:


Row 3 failed: No data available for request.
HTTP Status code: 204
Detailed response of server:




8it [00:01,  8.03it/s]

Row 6 failed: No data available for request.
HTTP Status code: 204
Detailed response of server:


Row 7 failed: No data available for request.
HTTP Status code: 204
Detailed response of server:




11it [00:02,  3.88it/s]

...

523it [10:46,  2.67it/s]

Row 522 failed: No data available for request.
HTTP Status code: 204
Detailed response of server:




531it [10:54,  1.26it/s]

Row 529 failed: No data available for request.
HTTP Status code: 204
Detailed response of server:


Row 530 failed: No data available for request.
HTTP Status code: 204
Detailed response of server:




532it [10:58,  1.24s/it]


## Finding all the unique seismic stations. 

To get an idea of what stations to consider and where to download the data from. 

In [91]:
# Filter out None before passing to np.unique
valid_stations = [s for s in su_cat['station'].values if s is not None]
unique_stations = np.unique(valid_stations)
print(unique_stations)

['B204' 'BRSP' 'CIHL' 'CLBH' 'CPCO' 'HIYU' 'HOA' 'HSR' 'LOO' 'LSON' 'MILD'
 'OBSR' 'PALM' 'PANH' 'RCS' 'REM' 'RER' 'SEP' 'SHUK' 'SHW' 'STAR' 'SUG'
 'TIMB' 'USFR' 'VALT' 'WIFE' 'YOCR']


## Downloading the data from 30 stations

In [163]:
starttime = obspy.UTCDateTime(2023, 7, 30, 1, 0, 0)
network = 'CC'
station = 'PANH'
channel = 'BH*'
location = '*'

stream = client.get_waveforms(network = network, 
                    station = station, 
                    channel = channel, 
                    location = loca, 
                    starttime = starttime,
                    endtime = starttime + 100)

In [156]:
model2.annotate(stream)

4 Trace(s) in Stream:
CC.PANH..SeismicCNN_eq | 2023-07-23T01:00:00.380000Z - 2023-07-23T01:00:00.380000Z | 50.0 s, 1 samples
CC.PANH..SeismicCNN_px | 2023-07-23T01:00:00.380000Z - 2023-07-23T01:00:00.380000Z | 50.0 s, 1 samples
CC.PANH..SeismicCNN_no | 2023-07-23T01:00:00.380000Z - 2023-07-23T01:00:00.380000Z | 50.0 s, 1 samples
CC.PANH..SeismicCNN_su | 2023-07-23T01:00:00.380000Z - 2023-07-23T01:00:00.380000Z | 50.0 s, 1 samples

In [176]:
starttime = obspy.UTCDateTime(2023, 7, 30, 1, 0, 0)


stations = [
    {"network": "UW", "station": "RER",  "channel": "HH?"},
    {"network": "UW", "station": "STAR",  "channel": "EH?"},
    {"network": "CC", "station": "PANH",  "channel": "BH?"},
    {"network": "CC", "station": "OBSR",  "channel": "BH?"}, 
    {"network": "CC", "station": "MILD",  "channel": "BH?"}, 
]



for sta in stations:
        i = 0
        prob_list = []
        time_index = []
        
        while i < 200:
    
            stream = client.get_waveforms(network = sta['network'], 
                            station = sta['station'], 
                            channel = sta['channel'], 
                            location = '*', 
                            starttime = starttime+i,
                            endtime = starttime + 100+i)
        
            prob_list.append(np.array(model2.annotate(stream)).flatten())
            time_index.append(i)
            i = i + 50
        
        np.savez_compressed(
        f"../dl_probs_csv/{sta['network']}_{sta['station']}_{starttime.date}.npz",
        times=np.array(time_index),
        probs=np.vstack(prob_list) if prob_list else np.empty((0, 4))
    )
        print(f"Saved {len(prob_list)} windows for {sta['network']}.{sta['station']}")

Saved 4 windows for UW.RER
Saved 4 windows for UW.STAR
Saved 4 windows for CC.PANH
Saved 4 windows for CC.OBSR
Saved 4 windows for CC.MILD


array([[1.0690180e-08, 2.1076683e-04, 9.9978906e-01, 6.1024224e-08],
       [1.0415563e-04, 5.5741198e-02, 9.1046524e-01, 3.3689383e-02],
       [5.3443760e-04, 1.5227825e-04, 9.9928170e-01, 3.1572035e-05],
       [5.7079078e-16, 1.3881865e-08, 2.4385240e-02, 9.7561479e-01]],
      dtype=float32)

In [137]:
#!/usr/bin/env python3
"""
Classify a month of continuous data for several stations in 100-second windows.
Outputs one CSV per station with the class probabilities per window.

Assumptions
-----------
* model2.annotate(Stream) → Stream with 4 Traces named
    SeismicCNN_eq/px/no/su (1 sample each = probability).
* Model expects 3-component, 50 Hz, 100 s Stream (5 000 samples/component).
* SeisBench or other preprocessing is handled **inside** `model2.annotate`
  or is not needed.  If you need custom preprocessing, insert it where marked.
"""

import os
import sys
import math
import csv
from datetime import timedelta

import obspy
from obspy import UTCDateTime, Stream
from obspy.clients.fdsn import Client
from tqdm import tqdm   # progress bar

# ────────────────────────── user config ──────────────────────────
START = UTCDateTime(2023, 6, 1)          # beginning of month (UTC)
END   = START + 2 * 3600           # ~30 days later
WINDOW_SEC   = 100                       # classifier input window
STRIDE_SEC = 50           # step between slice starts  ← NEW
CHUNK_SEC    = 3600                      # download one hour at a time
CLIENT       = Client("IRIS")            # or regional FDSN node
OUT_DIR      = "ml_probs_csv"            # output folder
os.makedirs(OUT_DIR, exist_ok=True)

stations = [
    {"network": "UW", "station": "RER",  "channel": "HH?"},
    {"network": "UW", "station": "STAR", "channel": "EH?"},
    {"network": "CC", "station": "PANH", "channel": "BH?"},
    {"network": "CC", "station": "OBSR", "channel": "BH?"},
    {"network": "CC", "station": "MILD", "channel": "BH?"},
]

# import or create your classifier here
# from my_models import model2
# model2.eval()
# ──────────────────────────────────────────────────────────────────


def classify_window(win_st: Stream):
    """
    Returns dict of class:prob for one 100-s 3-component Stream.
    """
    out_st = model2.annotate(win_st)
    # Expect 4 traces in the output stream (eq, px, no, su)
    return {tr.stats.channel.split("_")[-1]: float(tr.data[0]) for tr in out_st}


def slice_hour_to_windows(hour_st: Stream, chunk_start: UTCDateTime):
    """
    Yield (win_start, win_stream) for consecutive 100-s windows with 50-s stride
    **inside the current 1-hour chunk**.

    We skip any window whose *end* would spill past the chunk boundary.
    The first window of the *next* chunk will cover that spill-over area,
    so we avoid duplicate work.
    """
    sr         = 50.0  # Hz  (adjust if model trained on a different rate)
    n_samples  = int(sr * WINDOW_SEC)
    last_start = CHUNK_SEC - WINDOW_SEC  # 3600-100 = 3500 s

    for offset in range(0, last_start + 1, STRIDE_SEC):
        win_start = chunk_start + offset
        win_end   = win_start   + WINDOW_SEC

        # Pull a copy, pad with zeros if a tiny gap exists
        win = hour_st.copy().trim(win_start, win_end,
                                  pad=True, fill_value=0, nearest_sample=False)

        # Require exactly one Z, one N, one E trace and right sample count
        if len(win) >= 3 and all(len(tr) == n_samples for tr in win):
            yield win_start, win



def process_station(sta):
    """
    Stream-through-time loop for one station. Writes a CSV incrementally.
    """
    out_path = os.path.join(OUT_DIR,
                  f"{sta['network']}_{sta['station']}_{START.date}.csv")
    header = ["utc_start", "eq_prob", "px_prob", "no_prob", "su_prob"]

    with open(out_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)

        t = START
        total_hours = math.ceil((END - START) / CHUNK_SEC)
        with tqdm(total=total_hours,
                  desc=f"{sta['network']}.{sta['station']}",
                  unit="h") as pbar:
            while t < END:
                try:
                    hour_st = CLIENT.get_waveforms(
                        network   = sta["network"],
                        station   = sta["station"],
                        location  = "*",
                        channel   = sta["channel"],
                        starttime = t,
                        endtime   = t + CHUNK_SEC,
                        attach_response=False,
                    )
                    # OPTIONAL: pre-processing here (detrend, filter, …)

                    for win_start, win in slice_hour_to_windows(hour_st, t):
                        probs = classify_window(win)
                        writer.writerow([
                            win_start.isoformat(),
                            probs.get("eq", 0.0),
                            probs.get("px", 0.0),
                            probs.get("no", 0.0),
                            probs.get("su", 0.0)
                        ])

                except Exception as e:
                    # network hiccup or no data; skip hour
                    print(e)
                    pass

                t += CHUNK_SEC
                pbar.update(1)


# ───────────────────────────── main ──────────────────────────────
if __name__ == "__main__":
    for sta in stations:
        process_station(sta)

    print(f"\nFinished – CSVs saved in “{OUT_DIR}”.")


CC.MILD: 100%|██████████| 2/2 [00:04<00:00,  2.31s/h]


Finished – CSVs saved in “ml_probs_csv”.


In [ ]:
c